In [35]:
import io
import json
import pathlib

import folium
import pandas as pd
import numpy as np
import osr
import dateutil

In [26]:
# this is the file that was delivered by Rijkswaterstaat
# The file is a printed table, probably an extract from a database 
path = pathlib.Path('../../../data/rws/nap/historie/NAP_Historie.txt')
# open the file
stream = path.open()
# print the first few lines
print("".join(stream.readlines()[:5]))

+----------+-----------+-----------+----------+------------------------------------------------------------+-------------------------+-----------+
|ID        |x          |y          |project_id|titel                                                       |datum                    |NAP hoogte |
+----------+-----------+-----------+----------+------------------------------------------------------------+-------------------------+-----------+
|000A1011  | 163880.630| 540553.300|279H04    |HYDRO SCHARNEGOUTUM                                         |01-may-1986              |      5.121|
|000A1011  | 163880.630| 540553.300|332=04=NAP|SEC herziening 332w04 groningen friesland 1993              |28-jun-1993              |      5.122|



In [27]:
# because the table is not in a standardized format, we have some cleaning up to do
lines = []
for i, line in enumerate(path.open()):
    # split the first and third line (only dashes and +)
    if i in (0, 2):
        continue
    # strip the | and whitespace
    fields = line.split('|')
    # put all fields in a list and strip of reamining | 
    fields = [field.strip().strip('|') for field in fields]
    # remove first and last column (should be empty)
    assert fields[0] == '' and fields[-1] == ''
    fields = fields[1:-1]
    # rejoin with the | (some fields contain , and ;, so we separate by |)
    line = "|".join(fields)
    # keep a list
    lines.append(line)
# concatenate cleaned up fields
txt = "\n".join(lines)

In [31]:
# read the CSV file as a table
df = pd.read_csv(io.StringIO(txt), sep='|', dtype={
    'titel': str,
    'x': float,
    'y': float
})
# make sure all titles are strings (some floats)
df['titel'] = df['titel'].astype(str)
# convert dates to dates
# did not check if the date format is consistent (not a common date format), let the parser guess
df['date'] = df['datum'].apply(lambda x: dateutil.parser.parse(x))

# based on the instructions, everything with an equal sign or after 2005 should be the revised NAP
# TODO: check if NAP correction is consistent with correction in use by PSMSL to create a local tide gauge benchmark
def is_revised(row):
    if row['date'].year >= 2005:
        return True
    if '=' in row['project_id']:
        return True
    return False
df['revised'] = df.apply(is_revised, axis=1)

In [29]:
rd = osr.SpatialReference()
rd.ImportFromEPSG(28992)
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(4326)
rd2wgs84 = osr.CoordinateTransformation(rd, wgs84)

In [61]:
lnglatz = np.array(rd2wgs84.TransformPoints(np.c_[df.x, df.y]))
df['lon'] = lnglatz[:, 0]
df['lat'] = lnglatz[:, 1]
df['date_str'] = df['date'].apply(lambda x:x.isoformat())

In [119]:
grouped = df[np.logical_and(df.lat > 53.2, df.lon > 6.5)].groupby('ID')

records = []
for i, (key, group) in enumerate(iter(grouped)):
    if i > 500:
        break
    plot = {
      "title": {
        "text": str(key),
        "anchor": "start",
        "orient": "bottom"
      },
      "width": 300,
      "height": 200,
      "data": {"values": group[['date_str', 'NAP hoogte']].to_dict(orient='records')},
      "mark": "point",
      "encoding": {
        "x": {
          "field": "date_str",
          "type": "temporal",
          "title": "date " + key,
          "axis": {"format": "%Y"}
        },
        "y": {
          "field": "NAP hoogte",
          "type": "quantitative",
        
          "scale": {"domain": [group['NAP hoogte'].min(), group['NAP hoogte'].max()]}
            
        }
      }
    }
    records.append({
        "title": key,
        "lat": group.lat.iloc[0],
        "lon": group.lon.iloc[0],
        "plot": plot
    })


In [120]:
m = folium.Map(
    location=[52, 4],
    zoom_start=7,
    tiles='Stamen Terrain'
)

for record in records:
    folium.Marker(
        location=[record['lat'], record['lon']],
        popup=folium.Popup(max_width=450).add_child(
            folium.VegaLite(record['plot'], width=450, height=250))
    ).add_to(m)
m

In [127]:
df[np.in1d(df.ID, ['000A4041'])].sort_values('date') # , '000A3524'])]

,ID,x,y,project_id,titel,datum,NAP hoogte,date,revised,lon,lat,date_str
7602,000A4041,258010.5,594365.5,201,GR NAM SLOCHTEREN 78,01-jan-1978,4.423,1978-01-01,False,6.93326,53.325774,1978-01-01T00:00:00
7583,000A4041,258010.5,594365.5,272W05,WADDEN,01-jan-1981,4.402,1981-01-01,False,6.93326,53.325774,1981-01-01T00:00:00
7584,000A4041,258010.5,594365.5,231,GR NAM SLOCHTEREN 81,01-jan-1981,4.402,1981-01-01,False,6.93326,53.325774,1981-01-01T00:00:00
7587,000A4041,258010.5,594365.5,233W28,KL NAM SLOCHTEREN 82,01-sep-1982,4.395,1982-09-01,False,6.93326,53.325774,1982-09-01T00:00:00
7607,000A4041,258010.5,594365.5,247W12,KL NAM SLOCHTEREN 83,15-sep-1983,4.394,1983-09-15,False,6.93326,53.325774,1983-09-15T00:00:00
7582,000A4041,258010.5,594365.5,249W17,WP TBV OM HAVENSCHAP,04-oct-1983,4.395,1983-10-04,False,6.93326,53.325774,1983-10-04T00:00:00
7604,000A4041,258010.5,594365.5,273W01,KL NAM SLOCHTEREN 84,01-jan-1984,4.384,1984-01-01,False,6.93326,53.325774,1984-01-01T00:00:00
7605,000A4041,258010.5,594365.5,258W29,SEC.WP OTERDUM,01-apr-1984,4.394,1984-04-01,False,6.93326,53.325774,1984-04-01T00:00:00
7603,000A4041,258010.5,594365.5,281W11,KL NAM SLOCHTEREN 85,01-sep-1985,4.378,1985-09-01,False,6.93326,53.325774,1985-09-01T00:00:00
7586,000A4041,258010.5,594365.5,289W04,KL NAM SLOCHTEREN 86,01-sep-1986,4.367,1986-09-01,False,6.93326,53.325774,1986-09-01T00:00:00


In [124]:
df[df.titel == 'RIKZ metingen 2005 0A6038 en 05041']

,ID,x,y,project_id,titel,datum,NAP hoogte,date,revised,lon,lat,date_str
7444,000A4031,172350.51,513022.11,380=62,RIKZ metingen 2005 0A6038 en 05041,01-mar-2005,2.715,2005-03-01,True,5.643324,52.604472,2005-03-01T00:00:00
7622,000A4041,258010.50,594365.50,380=62,RIKZ metingen 2005 0A6038 en 05041,01-mar-2005,4.277,2005-03-01,True,6.933260,53.325774,2005-03-01T00:00:00
8299,000A5041,258000.00,594430.00,380=62,RIKZ metingen 2005 0A6038 en 05041,01-mar-2005,7.521,2005-03-01,True,6.933123,53.326355,2005-03-01T00:00:00
8482,000A6038,172409.08,513031.39,380=62,RIKZ metingen 2005 0A6038 en 05041,01-mar-2005,2.037,2005-03-01,True,5.644189,52.604553,2005-03-01T00:00:00
173541,020F0044,172360.00,513000.00,380=62,RIKZ metingen 2005 0A6038 en 05041,01-mar-2005,3.414,2005-03-01,True,5.643463,52.604273,2005-03-01T00:00:00
